<a href="https://colab.research.google.com/github/rajkumaricode/Live-Code/blob/main/Live_Code_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://github.com/cdr/code-server/blob/main/src/browser/media/pwa-icon-512.png?raw=true' height="200" alt="code-server"/></center>
#@markdown <center><h3>code-server<br />code-server is VS Code running on a remote server, accessible through the browser.</h3></center><br>
import os,sys, pathlib, zipfile, re, tarfile, shutil
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
REGION = "US" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
INSTALL_EXTENSION = "ms-python.python ritwickdey.LiveServer ms-vscode.cpptools"  # @param {type:"string"}
USER_DATA_DIR = "/content/tools/code-server/userdata" # @param {type:"string"}
OPEN_FOLDER = "/content/" #@param {type: "string"} 
TAG_NAME = "3.2.0" #@param {type: "string"} 
PACKAGES = "x86_64" #@param ["x86_64", "amd64"]
RUN_LATEST = True # @param {type:"boolean"}
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl,
    findPackageR,
    textAn
)

loadingAn()

os.makedirs('tools/code-server/', exist_ok=True)
os.makedirs('tools/temp', exist_ok=True)
binFile = ''

# Downloading code-server
if not os.path.exists("tools/code-server/README.md"):
  os.system("apt install net-tools -y")

  BASE_URL = r"https://github.com/cdr/code-server/"
  rawRdata = findPackageR("cdr/code-server",
                          f"linux-{PACKAGES}.tar.gz",
                          False if RUN_LATEST else TAG_NAME,
                          all_=True)
  file_name = rawRdata['assets']['name']
  urlF = rawRdata['assets']['browser_download_url']
  output_file = "tools/temp/code-server.tar.gz"

  textAn(f"Installing code-server {rawRdata['tag_name']} ...", ty="twg")
  
  urllib.request.urlretrieve(urlF, output_file)
  with tarfile.open(output_file, 'r:gz') as tar_ref:
      tar_ref.extractall('tools/temp/')
  os.renames("tools/temp/"+file_name[:-7], 'tools/code-server/')
  try:
    pathlib.Path(output_file).unlink()
  except FileNotFoundError:
    pass
  try:
    os.remove('tools/code-server/lib/libstdc++.so.6')
  except FileNotFoundError:
    pass
  
  binList = ['bin/code-server',
         'code-server']
  for b in binList:
    if os.path.exists('tools/code-server/'+b):
      binFile = b
      break
      
  # workspace settings
  configScript = """{
  "workbench.colorTheme": "Default Dark+",
  "editor.minimap.enabled": false
}
"""
  os.makedirs(f'{OPEN_FOLDER}/.vscode', exist_ok=True)
  with open(f'{OPEN_FOLDER}/.vscode/settings.json', 'w') as w:w.write(configScript)

  if INSTALL_EXTENSION:
    perExtension = INSTALL_EXTENSION.split(' ')
    for l in perExtension:
      cmdE = f"./{binFile} " \
      f"--user-data-dir {USER_DATA_DIR}" \
      f" --install-extension {l}"
      runSh(cmdE, cd="tools/code-server", shell=True)


if not findProcess("node", "--extensions-dir"):
  cmdDo = f"./{binFile} --auth none " \
    f" --port 5050 --user-data-dir {USER_DATA_DIR}" \
    " &"
  runSh(cmdDo, 
        cd="tools/code-server",
        shell=True)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['code-server', 5050, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/code-server.yml", 30499]
).start('code-server', displayB=False)
displayUrl(server, EcUrl=f"/?folder={OPEN_FOLDER}")